TRAINING DATASET USADO ES LFW (Labeled Faces in the Wild https://vis-www.cs.umass.edu/lfw/ UMASS)

INFERENCING DATASETS USED 

CFP DATASET, CELEBRITIES DATASET FACES FRONTAL AND PROFILE http://www.cfpw.io/ 

MIT CIBCL FACE RECOGNITION, SEVERAL CLASSES http://cbcl.mit.edu/software-datasets/heisele/facerecognition-database.html

JEFF Japanese Female Facial expression FACE RECOGITION JAPAN https://zenodo.org/records/3451524

CROSS AGE CELEBRITY DATASET https://www.v7labs.com/open-datasets/cacd 

In [1]:
import os
import random
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import pandas as pd
from glob import glob

In [2]:
#SEEDING
import random
random.seed(42)  # Set the seed to a fixed number


In [3]:
torch.manual_seed(42)  # Set the seed for PyTorch


In [4]:
print(torch.__version__)  #1.11.0+cpu
print(torch.cuda.is_available())


2.2.0+cu121
True


In [5]:
if torch.cuda.device_count() > 1:
    print(f"{torch.cuda.device_count()} GPUs available")
else:
    print("Single GPU or no GPU available")


4 GPUs available


In [6]:

# Check if GPU is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)



cuda


In [7]:

# If using CUDA (PyTorch)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)  # if you are using multi-GPU.


In [8]:
# Enforce deterministic behavior in PyTorch (optional, may impact performance)
# This is important for reproducibility, especially for certain operations on GPUs
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [9]:
#!pip install opencv-python-headless
os.getcwd()

'/home/rapids/notebooks/data/storage/slima'

In [10]:
# Change the present working directory
os.chdir('/home/rapids/notebooks/data/storage/slima')


In [11]:
embedding_size=512
n_classes=1
#normalization size in pixels
resizex=112
resizey= 112

In [12]:
# # Set the directory path
# base_dir = r'D:\JEFF face recognition db Japan\jaffedbase'
# output_dir = r'D:\JEFF face recognition db Japan\jaffedbase_obfuscated'
# state_dict_path = r'D:\JEFF face recognition db Japan\fuzzyarcface_resnet101_customized5  tau 0.9 100 epochs fuzzymembership raw mask ones.pth'
# arcface_state_dict_path = r'path\to\arcface_resnet101_customized4_tau_0.5_100_epochs.pth'


# Set the directory path
base_dir = 'jaffedbase'
output_dir = 'jaffedbase_obfuscated'
fuzzyarcface_state_dict_path = 'fuzzyarcface_resnet101_customized5  tau 0.9 100 epochs fuzzymembership raw mask ones.pth'
arcface_state_dict_path = 'arcface_resnet101_customized4 tau 0.5 100 epochs.pth'


# Set the directory path
base_dir = 'jaffedbase'
output_dir = 'JEFF face recognition db Japan\jaffedbase_obfuscated'


fuzzyarcface1_state_dict_path = 'fuzzyarcface_resnet101_customized5  tau 0.9 100 epochs fuzzymembership raw mask ones.pth'
fuzzyarcface2_state_dict_path = 'fuzzyarcface_resnet101_customized4 tau 0.5 fuzzymembership raw mask ones.pth'
fuzzyarcface3_state_dict_path = 'fuzzyarcface_resnet101_customized4 tau 0.1 100 epochs.pth'


arcface_state_dict_path = 'arcface_resnet101_customized4 tau 0.5 100 epochs.pth'



In [13]:
# Define the model creation function
def create_base_model():
    model = iresnet100()  # Ensure this is your actual model class
    return model

# Remove 'module.' prefix from keys if necessary
def remove_module_prefix(state_dict):
    new_state_dict = {}
    for k, v in state_dict.items():
        if k.startswith('module.'):
            new_state_dict[k[7:]] = v  # Remove 'module.' prefix
        else:
            new_state_dict[k] = v
    return new_state_dict


# # Define models for each training method
# def create_base_model():
#     model = iresnet100()
#     if torch.cuda.device_count() > 1:
#         model = torch.nn.DataParallel(model)
#     return model

# Assuming you have a function to remove 'module.' prefix if present
# def remove_module_prefix(state_dict):
#     return {k.replace('module.', ''): v for k, v in state_dict.items()}
# def remove_module_prefix(state_dict):
#     new_state_dict = {}
#     for k, v in state_dict.items():
#         if k.startswith('module.'):
#             new_state_dict[k[7:]] = v  # Remove 'module.' prefix
#         else:
#             new_state_dict[k] = v
#     return new_state_dict


# Function to obfuscate an image
def obfuscate_image(image, obfuscation_level):
    h, w = image.shape
    mask = np.random.rand(h, w) < obfuscation_level
    obfuscated_image = image.copy()
    obfuscated_image[mask] = 0  # Set the obfuscated pixels to black (or you can use any other obfuscation method)
    return obfuscated_image

# Function to extract embedding using the model
def extract_embedding(model, image, device):
    model.eval()
    with torch.no_grad():
        image = transform(image).unsqueeze(0).to(device)
        embedding = model(image).cpu().numpy().flatten()
    return embedding

# Function to compute cosine similarity
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))



In [14]:
#iRESNET100 IMPLEMENTATION


__all__ = ['iresnet18', 'iresnet34', 'iresnet50', 'iresnet100', 'iresnet200']
using_ckpt = False

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=dilation,
                     groups=groups,
                     bias=False,
                     dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size=1,
                     stride=stride,
                     bias=False)


class IBasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None,
                 groups=1, base_width=64, dilation=1):
        super(IBasicBlock, self).__init__()
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        self.bn1 = nn.BatchNorm2d(inplanes, eps=1e-05,)
        self.conv1 = conv3x3(inplanes, planes)
        self.bn2 = nn.BatchNorm2d(planes, eps=1e-05,)
        self.prelu = nn.PReLU(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn3 = nn.BatchNorm2d(planes, eps=1e-05,)
        self.downsample = downsample
        self.stride = stride

    def forward_impl(self, x):
        identity = x
        out = self.bn1(x)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn3(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return out        

    def forward(self, x):
        if self.training and using_ckpt:
            return checkpoint(self.forward_impl, x)
        else:
            return self.forward_impl(x)


class IResNet(nn.Module):
    #fc_scale = 7 * 7  * 4  #for dim 224x 224
    fc_scale = 7 * 7    #for dim 112x 112
    
    def __init__(self,
                 block, layers, dropout=0, embedding_size=embedding_size, num_features=n_classes, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None, fp16=False):
        super(IResNet, self).__init__()
        self.extra_gflops = 0.0
        self.fp16 = fp16
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes, eps=1e-05)
        self.prelu = nn.PReLU(self.inplanes)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=2)
        self.layer2 = self._make_layer(block,
                                       128,
                                       layers[1],
                                       stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block,
                                       256,
                                       layers[2],
                                       stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block,
                                       512,
                                       layers[3],
                                       stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.bn2 = nn.BatchNorm2d(512 * block.expansion, eps=1e-05,)
        self.dropout = nn.Dropout(p=dropout, inplace=True)
        self.fc = nn.Linear(512 * block.expansion * self.fc_scale, embedding_size)
        self.features = nn.BatchNorm1d(embedding_size, eps=1e-05)
        nn.init.constant_(self.features.weight, 1.0)
        self.features.weight.requires_grad = False

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, 0, 0.1)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, IBasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion, eps=1e-05, ),
            )
        layers = []
        layers.append(
            block(self.inplanes, planes, stride, downsample, self.groups,
                  self.base_width, previous_dilation))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(self.inplanes,
                      planes,
                      groups=self.groups,
                      base_width=self.base_width,
                      dilation=self.dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        with torch.cuda.amp.autocast(self.fp16):
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.prelu(x)
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = self.bn2(x)
            x = torch.flatten(x, 1)
            x = self.dropout(x)
        x = self.fc(x.float() if self.fp16 else x)
        x = self.features(x)
        return x


def _iresnet(arch, block, layers, pretrained, progress, **kwargs):
    model = IResNet(block, layers, **kwargs)
    if pretrained:
        raise ValueError()
    return model


def iresnet18(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet18', IBasicBlock, [2, 2, 2, 2], pretrained,
                    progress, **kwargs)


def iresnet34(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet34', IBasicBlock, [3, 4, 6, 3], pretrained,
                    progress, **kwargs)


def iresnet50(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet50', IBasicBlock, [3, 4, 14, 3], pretrained,
                    progress, **kwargs)


def iresnet100(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet100', IBasicBlock, [3, 13, 30, 3], pretrained,
                    progress, **kwargs)


def iresnet200(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet200', IBasicBlock, [6, 26, 60, 6], pretrained,
                    progress, **kwargs)

In [15]:
# import torch

# # Load the state_dict
# fuzzyarcface1_state_dict = torch.load(fuzzyarcface1_state_dict_path, map_location=device)

# # Remove the 'module.' prefix if necessary
# fuzzyarcface1_state_dict = remove_module_prefix(fuzzyarcface1_state_dict)

# fuzzyarcface1_model = create_base_model()

# # Load the state_dict into the model
# fuzzyarcface1_model.load_state_dict(fuzzyarcface1_state_dict)

# # Move the model to the device
# fuzzyarcface1_model.to(device)

# # Wrap the model in DataParallel if multiple GPUs are available
# if torch.cuda.device_count() > 1:
#     fuzzyarcface1_model = torch.nn.DataParallel(fuzzyarcface1_model)

In [16]:

# Create output directory if not exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the FuzzyArcface1 model
#device = torch.device('gpu')
fuzzyarcface1_model = create_base_model()
fuzzyarcface1_state_dict = torch.load(fuzzyarcface1_state_dict_path, map_location=device)
fuzzyarcface1_state_dict = remove_module_prefix(fuzzyarcface1_state_dict)
fuzzyarcface1_model.load_state_dict(fuzzyarcface1_state_dict)
fuzzyarcface1_model.to(device)
# Wrap the model in DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    fuzzyarcface1_model = torch.nn.DataParallel(fuzzyarcface1_model)
    
# Load the Arcface model
arcface_model = create_base_model()
arcface_state_dict = torch.load(arcface_state_dict_path, map_location=device)
arcface_state_dict = remove_module_prefix(arcface_state_dict)
arcface_model.load_state_dict(arcface_state_dict)
arcface_model.to(device)
# Wrap the model in DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    arcface_model = torch.nn.DataParallel(arcface_model)
    
# Load the FuzzyArcface2 model
fuzzyarcface2_model = create_base_model()
fuzzyarcface2_state_dict = torch.load(fuzzyarcface2_state_dict_path, map_location=device)
fuzzyarcface2_state_dict = remove_module_prefix(fuzzyarcface2_state_dict)
fuzzyarcface2_model.load_state_dict(fuzzyarcface2_state_dict)
fuzzyarcface2_model.to(device)
# Wrap the model in DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    fuzzyarcface2_model = torch.nn.DataParallel(fuzzyarcface2_model)

# Load the FuzzyArcface3 model
fuzzyarcface3_model = create_base_model()
fuzzyarcface3_state_dict = torch.load(fuzzyarcface3_state_dict_path, map_location=device)
fuzzyarcface3_state_dict = remove_module_prefix(fuzzyarcface3_state_dict)
fuzzyarcface3_model.load_state_dict(fuzzyarcface3_state_dict)
fuzzyarcface3_model.to(device)
# Wrap the model in DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    fuzzyarcface3_model = torch.nn.DataParallel(fuzzyarcface3_model)


In [17]:

# Define the transform
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
])


In [18]:

# Get all image paths
image_paths = glob(os.path.join(base_dir, '*.tiff'))

# Dictionary to store class-wise images
class_images = {}

# Read images and group by class
for image_path in image_paths:
    image_name = os.path.basename(image_path)
    class_name = image_name.split('.')[0]
    if class_name not in class_images:
        class_images[class_name] = []
    class_images[class_name].append(image_path)


In [19]:

# Fixed obfuscation levels
obfuscation_levels = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30]

# DataFrame to store results
results_df = pd.DataFrame(columns=[
    'Class', 'Obfuscation Level',
    'FuzzyArcFace1 Similarity', 'ArcFace Similarity',
    'FuzzyArcFace2 Similarity', 'FuzzyArcFace3 Similarity'
])

In [20]:
# Process each class
for class_name, images in class_images.items():
    # Randomly pick one image per class
    selected_image_path = random.choice(images)
    selected_image = cv2.imread(selected_image_path, cv2.IMREAD_GRAYSCALE)

    # Create class directory in the output directory
    class_output_dir = os.path.join(output_dir, class_name)
    if not os.path.exists(class_output_dir):
        os.makedirs(class_output_dir)

    # Save the original image in the class directory
    original_image_name = os.path.basename(selected_image_path)
    cv2.imwrite(os.path.join(class_output_dir, original_image_name), selected_image)

    # Convert the original image to RGB PIL format
    pil_image = Image.fromarray(cv2.cvtColor(selected_image, cv2.COLOR_GRAY2RGB))

    # Extract embeddings for the original image
    original_fuzzyarcface1_embedding = extract_embedding(fuzzyarcface1_model, pil_image, device)
    original_arcface_embedding = extract_embedding(arcface_model, pil_image, device)
    original_fuzzyarcface2_embedding = extract_embedding(fuzzyarcface2_model, pil_image, device)
    original_fuzzyarcface3_embedding = extract_embedding(fuzzyarcface3_model, pil_image, device)

    # Generate and save obfuscated images, and compute similarity
    for obfuscation_level in obfuscation_levels:
        obfuscated_image = obfuscate_image(selected_image, obfuscation_level)
        obfuscated_image_name = f"{class_name}.obf{int(obfuscation_level*100)}.png"
        cv2.imwrite(os.path.join(class_output_dir, obfuscated_image_name), obfuscated_image)

        # Convert the obfuscated image to RGB PIL format
        pil_obfuscated_image = Image.fromarray(cv2.cvtColor(obfuscated_image, cv2.COLOR_GRAY2RGB))

        # Extract embeddings for the obfuscated image
        obfuscated_fuzzyarcface1_embedding = extract_embedding(fuzzyarcface1_model, pil_obfuscated_image, device)
        obfuscated_arcface_embedding = extract_embedding(arcface_model, pil_obfuscated_image, device)
        obfuscated_fuzzyarcface2_embedding = extract_embedding(fuzzyarcface2_model, pil_obfuscated_image, device)
        obfuscated_fuzzyarcface3_embedding = extract_embedding(fuzzyarcface3_model, pil_obfuscated_image, device)

        # Compute cosine similarities
        fuzzyarcface1_similarity = cosine_similarity(original_fuzzyarcface1_embedding, obfuscated_fuzzyarcface1_embedding)
        arcface_similarity = cosine_similarity(original_arcface_embedding, obfuscated_arcface_embedding)
        fuzzyarcface2_similarity = cosine_similarity(original_fuzzyarcface2_embedding, obfuscated_fuzzyarcface2_embedding)
        fuzzyarcface3_similarity = cosine_similarity(original_fuzzyarcface3_embedding, obfuscated_fuzzyarcface3_embedding)
        
        # Append results to DataFrame
        results_df = pd.concat([results_df, pd.DataFrame([{
            'Class': class_name,
            'Obfuscation Level': obfuscation_level * 100,
            'FuzzyArcFace1 Similarity': fuzzyarcface1_similarity * 100,
            'ArcFace Similarity': arcface_similarity * 100,
            'FuzzyArcFace2 Similarity': fuzzyarcface2_similarity * 100,
            'FuzzyArcFace3 Similarity': fuzzyarcface3_similarity * 100
        }])], ignore_index=True)



In [21]:
# Print the results
print(results_df)

# Save results to a CSV file
results_df.to_csv(os.path.join(output_dir, 'similarity_results_fuzzyall_crisp2.csv'), index=False)

print("Processing complete.")

Empty DataFrame
Columns: [Class, Obfuscation Level, FuzzyArcFace1 Similarity, ArcFace Similarity, FuzzyArcFace2 Similarity, FuzzyArcFace3 Similarity]
Index: []
Processing complete.


CFP DATASET

Load the images and randomly select one per class.
Apply obfuscation at different levels.
Use a pretrained model to extract embeddings and calculate similarity.
Output the results in a CSV file.

In [22]:

# Set paths and device
base_dir_cfp = 'CFP DATASET/Data/Images'
output_dir = 'CFP DATASET/Processed'

In [23]:

# Get all image paths
class_folders = [os.path.join(base_dir_cfp, d) for d in os.listdir(base_dir_cfp) if os.path.isdir(os.path.join(base_dir_cfp, d))]

# Prepare results dataframe
results_df = pd.DataFrame(columns=['Class', 'Obfuscation Level', 'FuzzyArcFace Similarity', 'ArcFace Similarity'])


FileNotFoundError: [Errno 2] No such file or directory: 'CFP DATASET/Data/Images'

In [24]:
# Function to obfuscate image
def obfuscate_image(image, obfuscation_level):
    draw = ImageDraw.Draw(image)
    width, height = image.size
    num_obfuscation_points = int(obfuscation_level * width * height)
    for _ in range(num_obfuscation_points):
        x = random.randint(0, width - 1)
        y = random.randint(0, height - 1)
        draw.point((x, y), fill="black")
    return image

In [25]:
from PIL import Image, ImageDraw

In [ ]:

for class_folder in class_folders:
    class_name = os.path.basename(class_folder)
    frontal_folder = os.path.join(class_folder, 'frontal')
    image_files = glob(os.path.join(frontal_folder, '*.jpg'))

    # Check if image_files list is empty
    if not image_files:
        print(f"No images found in folder: {frontal_folder}")
        continue

    selected_image_path = random.choice(image_files)
    #selected_image = cv2.imread(selected_image_path, cv2.IMREAD_GRAYSCALE)
    #pil_image = Image.fromarray(cv2.cvtColor(selected_image, cv2.COLOR_BGR2RGB))
    selected_image = cv2.imread(selected_image_path, cv2.IMREAD_COLOR)


    
    # Create class directory in the output directory
    class_output_dir = os.path.join(output_dir, class_name)
    if not os.path.exists(class_output_dir):
        os.makedirs(class_output_dir)

    # Save the original image in the class directory
    original_image_name = os.path.basename(selected_image_path)
    cv2.imwrite(os.path.join(class_output_dir, original_image_name), selected_image)

    # Convert the original image to RGB PIL format
    #pil_image = Image.fromarray(cv2.cvtColor(selected_image, cv2.COLOR_GRAY2RGB))
       # Convert the original image to RGB PIL format
    pil_image = Image.fromarray(cv2.cvtColor(selected_image, cv2.COLOR_BGR2RGB))

    
    
    # Extract embeddings for the original image
    original_fuzzyarcface1_embedding = extract_embedding(fuzzyarcface1_model, pil_image, device)
    original_arcface_embedding = extract_embedding(arcface_model, pil_image, device)
    original_fuzzyarcface2_embedding = extract_embedding(fuzzyarcface2_model, pil_image, device)
    original_fuzzyarcface3_embedding = extract_embedding(fuzzyarcface3_model, pil_image, device)

    # Obfuscation levels
    #obfuscation_levels = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30]

    # Generate and save obfuscated images, and compute similarity
    for obfuscation_level in obfuscation_levels:
        obfuscated_image = obfuscate_image(pil_image.copy(), obfuscation_level)
        obfuscated_image_name = f"{class_name}.obf{int(obfuscation_level*100)}.png"
        obfuscated_image.save(os.path.join(class_output_dir, obfuscated_image_name))

        # Extract embeddings for the obfuscated image
        obfuscated_fuzzyarcface1_embedding = extract_embedding(fuzzyarcface1_model, obfuscated_image, device)
        obfuscated_arcface_embedding = extract_embedding(arcface_model, obfuscated_image, device)
        obfuscated_fuzzyarcface2_embedding = extract_embedding(fuzzyarcface2_model, obfuscated_image, device)
        obfuscated_fuzzyarcface3_embedding = extract_embedding(fuzzyarcface3_model, obfuscated_image, device)

#         # Compute cosine similarities
#         fuzzyarcface1_similarity = cosine_similarity([original_fuzzyarcface1_embedding], [obfuscated_fuzzyarcface1_embedding])[0][0] * 100
#         arcface_similarity = cosine_similarity([original_arcface_embedding], [obfuscated_arcface_embedding])[0][0] * 100
#         fuzzyarcface2_similarity = cosine_similarity([original_fuzzyarcface2_embedding], [obfuscated_fuzzyarcface2_embedding])[0][0] * 100
#         fuzzyarcface3_similarity = cosine_similarity([original_fuzzyarcface3_embedding], [obfuscated_fuzzyarcface3_embedding])[0][0] * 100
        # Compute cosine similarities
        fuzzyarcface1_similarity = cosine_similarity(original_fuzzyarcface1_embedding, obfuscated_fuzzyarcface1_embedding)
        arcface_similarity = cosine_similarity(original_arcface_embedding, obfuscated_arcface_embedding)
        fuzzyarcface2_similarity = cosine_similarity(original_fuzzyarcface2_embedding, obfuscated_fuzzyarcface2_embedding)
        fuzzyarcface3_similarity = cosine_similarity(original_fuzzyarcface3_embedding, obfuscated_fuzzyarcface3_embedding)
        


        # Append results to DataFrame
        results_df = pd.concat([results_df, pd.DataFrame([{
            'Class': class_name,
            'Obfuscation Level': obfuscation_level * 100,
            'FuzzyArcFace1 Similarity': fuzzyarcface1_similarity,
            'ArcFace Similarity': arcface_similarity,
            'FuzzyArcFace2 Similarity': fuzzyarcface2_similarity,
            'FuzzyArcFace3 Similarity': fuzzyarcface3_similarity
        }])], ignore_index=True)



In [ ]:
# Save results to CSV
results_df.to_csv('cfp_similarity_results.csv', index=False)

print("Analysis complete for CFP celebrities face db. Results saved to 'cfp_similarity_results.csv'.")

CROSS AGE CELEBRITY DATASET https://www.v7labs.com/open-datasets/cacd

For each class, compute the embeddings for all images.
Compare each image's embeddings with every other image's embeddings within the same class.
Calculate and store the cosine similarity for each comparison.

In [26]:
# Set paths and device
base_dir_cacd = 'CACD2000/CACD2000'
output_dir = 'CACD2000/CACD2000/Processed'


In [27]:
# Prepare results dataframe
results_df = pd.DataFrame(columns=['Class', 'Image1', 'Image2', 'FuzzyArcFace1 Similarity', 'ArcFace Similarity', 'FuzzyArcFace2 Similarity', 'FuzzyArcFace3 Similarity'])

# Get all image paths
image_files = glob(os.path.join(base_dir_cacd, '*.jpg'))


In [28]:
# Group images by celebrity name (class)
class_images = {}
for image_path in image_files:
    image_name = os.path.basename(image_path)
    class_name = image_name.split('_')[1]  # Assuming the format is <id>_<celebrity_name>_<other>.jpg
    if class_name not in class_images:
        class_images[class_name] = []
    class_images[class_name].append(image_path)


In [ ]:
# Process each class
for class_name, images in class_images.items():
    if len(images) < 2:
        print(f"Not enough images for class: {class_name}")
        continue

    embeddings = {'FuzzyArcFace1': [], 'ArcFace': [], 'FuzzyArcFace2': [], 'FuzzyArcFace3': []}
    image_names = []

    # Compute embeddings for all images in the class
    for image_path in images:
        image_name = os.path.basename(image_path)
        selected_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
                        # Check if the image was read correctly
        if selected_image is None:
            print(f"Error reading image: {image_path}")
            continue
            
        pil_image = Image.fromarray(cv2.cvtColor(selected_image, cv2.COLOR_BGR2RGB))
        
        # Extract embeddings
        fuzzyarcface1_embedding = extract_embedding(fuzzyarcface1_model, pil_image, device)
        arcface_embedding = extract_embedding(arcface_model, pil_image, device)
        fuzzyarcface2_embedding = extract_embedding(fuzzyarcface2_model, pil_image, device)
        fuzzyarcface3_embedding = extract_embedding(fuzzyarcface3_model, pil_image, device)

        embeddings['FuzzyArcFace1'].append(fuzzyarcface1_embedding)
        embeddings['ArcFace'].append(arcface_embedding)
        embeddings['FuzzyArcFace2'].append(fuzzyarcface2_embedding)
        embeddings['FuzzyArcFace3'].append(fuzzyarcface3_embedding)
        image_names.append(image_name)
    
    # Compare each image with every other image in the same class
    for i in range(len(images)):
        for j in range(i + 1, len(images)):
            image1_name = image_names[i]
            image2_name = image_names[j]

            # Compute cosine similarities
            fuzzyarcface1_similarity = cosine_similarity(embeddings['FuzzyArcFace1'][i], embeddings['FuzzyArcFace1'][j]) * 100
            arcface_similarity = cosine_similarity(embeddings['ArcFace'][i], embeddings['ArcFace'][j]) * 100
            fuzzyarcface2_similarity = cosine_similarity(embeddings['FuzzyArcFace2'][i], embeddings['FuzzyArcFace2'][j]) * 100
            fuzzyarcface3_similarity = cosine_similarity(embeddings['FuzzyArcFace3'][i], embeddings['FuzzyArcFace3'][j]) * 100

            # Append results to DataFrame
            results_df = pd.concat([results_df, pd.DataFrame([{
                'Class': class_name,
                'Image1': image1_name,
                'Image2': image2_name,
                'FuzzyArcFace1 Similarity': fuzzyarcface1_similarity,
                'ArcFace Similarity': arcface_similarity,
                'FuzzyArcFace2 Similarity': fuzzyarcface2_similarity,
                'FuzzyArcFace3 Similarity': fuzzyarcface3_similarity
            }])], ignore_index=True)

In [ ]:
# Save results to CSV
results_df.to_csv('cacd2000_similarity_results.csv', index=False)

print("Analysis complete for CACD2000 celebrities face db. Results saved to 'cacd2000_similarity_results.csv'.")

MIT CIBCL FACE RECOGNITION DATASET